In [1]:
!nvidia-smi

Fri Aug 29 16:45:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install conllu

In [5]:
from conllu import parse
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel
from transformers.optimization import get_linear_schedule_with_warmup
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pickle
import json
import random

In [ ]:
!mkdir -p /content/drive/MyDrive/coreference/
!ls /content/drive/MyDrive/coreference/

coreference_model.pth  Tamil-coreference.ipynb	web-ui.ipynb
merged4.conll	       training.ipynb


In [6]:
with open("/content/drive/MyDrive/coreference/merged4.conll", "r", encoding="utf-8") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]
lines[10:20]

['tamil_doc\t000\t9\tவந்திருக்கிறவன்\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t10\tஎன்று\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t11\tநேற்றைக்கே\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t12\tஊகித்தேன்\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t13\t.\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t14\tஇன்றைக்குக்\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t15\tகாலையில்\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t16\tஉன்\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t17\tசிநேகிதன்\t-\t-\t-\tSpeaker#1\t-\t-',
 'tamil_doc\t000\t18\t,\t-\t-\t-\tSpeaker#1\t-\t-']

In [7]:
# Parse CoNLL format
sentences = []
current_sentence = []
all_tokens = []

for line in lines:
    line = line.strip()
    if line.startswith("#") or not line:
        if current_sentence:
            sentences.append(current_sentence)
            current_sentence = []
        continue
    parts = line.split('\t')
    if len(parts) >= 4:
        token = parts[3]
        current_sentence.append(token)
        all_tokens.append(token)

if current_sentence:
    sentences.append(current_sentence)

print(f"Total tokens: {len(all_tokens)}")
print(f"Total sentences: {len(sentences)}")

# Create limited training data pairs (memory efficient)
def create_limited_pairs(all_tokens, max_pairs=5000):
    pairs = []
    labels = []

    # Count token frequencies
    token_counts = {}
    for token in all_tokens:
        token_counts[token] = token_counts.get(token, 0) + 1

    unique_tokens = list(token_counts.keys())
    print(f"Unique tokens: {len(unique_tokens)}")

    # Create positive pairs (same tokens that appear multiple times)
    pos_pairs = []
    for token in unique_tokens:
        if token_counts[token] > 1:  # Token appears multiple times
            pos_pairs.append((token, token))

    print(f"Potential positive pairs: {len(pos_pairs)}")

    # If no repeated tokens, create some positive pairs from same tokens
    if len(pos_pairs) == 0:
        for token in unique_tokens[:100]:  # Take first 100 tokens
            pos_pairs.append((token, token))

    # Limit positive pairs
    pos_pairs = pos_pairs[:min(len(pos_pairs), max_pairs//2)]

    # Create negative pairs (different tokens)
    neg_pairs = []
    random.seed(42)

    target_neg = min(len(pos_pairs), max_pairs//2)
    if target_neg == 0:
        target_neg = max_pairs//2

    attempts = 0
    while len(neg_pairs) < target_neg and attempts < 10000:
        token1 = random.choice(unique_tokens)
        token2 = random.choice(unique_tokens)
        if token1 != token2 and (token1, token2) not in neg_pairs:
            neg_pairs.append((token1, token2))
        attempts += 1

    # Combine pairs
    pairs = pos_pairs + neg_pairs
    labels = [1] * len(pos_pairs) + [0] * len(neg_pairs)

    return pairs, labels

pairs, labels = create_limited_pairs(all_tokens)
print(f"Total pairs: {len(pairs)}, Positive pairs: {sum(labels)}")

if len(pairs) == 0:
    print("No pairs generated. Creating minimal dataset...")
    # Create minimal dataset
    unique_tokens = list(set(all_tokens))[:20]
    pairs = [(unique_tokens[i], unique_tokens[i]) for i in range(min(10, len(unique_tokens)))]
    pairs += [(unique_tokens[i], unique_tokens[j]) for i in range(min(5, len(unique_tokens))) for j in range(i+1, min(10, len(unique_tokens)))]
    labels = [1] * 10 + [0] * (len(pairs) - 10)

# Train-test split (using same data as requested)
if len(set(labels)) > 1:
    train_pairs, test_pairs, train_labels, test_labels = train_test_split(
        pairs, labels, test_size=0.2, random_state=42, stratify=labels
    )
else:
    train_pairs, test_pairs, train_labels, test_labels = train_test_split(
        pairs, labels, test_size=0.2, random_state=42
    )

print(f"Train pairs: {len(train_pairs)}, Test pairs: {len(test_pairs)}")

Total tokens: 126954
Total sentences: 361
Unique tokens: 26076
Potential positive pairs: 10285
Total pairs: 5000, Positive pairs: 2500
Train pairs: 4000, Test pairs: 1000


In [ ]:
# Returns a dictionary with model-ready input

class CoreferenceDataset(Dataset):
    def __init__(self, pairs, labels, tokenizer, max_length=128):
        self.pairs = pairs
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]
        label = self.labels[idx]

        text = f"{pair[0]} [SEP] {pair[1]}"

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Fine-tunes a BERT model to classify Tamil text pairs as either coreferent or not.

class CoreferenceModel(nn.Module):
    def __init__(self, model_name):
        super(CoreferenceModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.classifier(output)

In [ ]:
# training pipeline: it loads data into batches, defines the optimizer, loss, and learning rate scheduler for fine-tuning the BERT-based coreference model.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'google/muril-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = CoreferenceModel(model_name).to(device)

train_dataset = CoreferenceDataset(train_pairs, train_labels, tokenizer)
test_dataset = CoreferenceDataset(test_pairs, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

total_steps = len(train_loader) * 3  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/953M [00:00<?, ?B/s]

In [ ]:


def train_epoch(model, data_loader, optimizer, device, scheduler):
    model = model.train()
    losses = []

    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()

    return np.mean(losses)

# Training loop
for epoch in range(10):
    train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    print(f'Epoch {epoch+1}/3, Train Loss: {train_loss:.4f}')

model.safetensors:   0%|          | 0.00/953M [00:00<?, ?B/s]

Epoch 1/3, Train Loss: 0.4200
Epoch 2/3, Train Loss: 0.1069
Epoch 3/3, Train Loss: 0.0473
Epoch 4/3, Train Loss: 0.0418
Epoch 5/3, Train Loss: 0.0430
Epoch 6/3, Train Loss: 0.0403
Epoch 7/3, Train Loss: 0.0432
Epoch 8/3, Train Loss: 0.0423
Epoch 9/3, Train Loss: 0.0427
Epoch 10/3, Train Loss: 0.0433


In [11]:
def calculate_muc(true_clusters, pred_clusters):
    def num_links(clusters):
        return sum(len(cluster) - 1 for cluster in clusters if len(cluster) > 1)

    true_links = num_links(true_clusters)
    pred_links = num_links(pred_clusters)

    if pred_links == 0:
        return 0.0, 0.0, 0.0

    common_links = 0
    for pred_cluster in pred_clusters:
        for true_cluster in true_clusters:
            intersection = len(set(pred_cluster) & set(true_cluster))
            if intersection > 1:
                common_links += intersection - 1

    precision = common_links / pred_links if pred_links > 0 else 0
    recall = common_links / true_links if true_links > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

def calculate_bcubed(true_clusters, pred_clusters):
    entity_to_true_cluster = {}
    entity_to_pred_cluster = {}

    for i, cluster in enumerate(true_clusters):
        for entity in cluster:
            entity_to_true_cluster[entity] = i

    for i, cluster in enumerate(pred_clusters):
        for entity in cluster:
            entity_to_pred_cluster[entity] = i

    precisions = []
    recalls = []

    all_entities = set(entity_to_true_cluster.keys()) | set(entity_to_pred_cluster.keys())

    for entity in all_entities:
        true_cluster_id = entity_to_true_cluster.get(entity, -1)
        pred_cluster_id = entity_to_pred_cluster.get(entity, -1)

        if true_cluster_id >= 0:
            true_cluster = true_clusters[true_cluster_id]
        else:
            true_cluster = {entity}

        if pred_cluster_id >= 0:
            pred_cluster = pred_clusters[pred_cluster_id]
        else:
            pred_cluster = {entity}

        intersection = len(set(true_cluster) & set(pred_cluster))

        precision = intersection / len(pred_cluster)
        recall = intersection / len(true_cluster)

        precisions.append(precision)
        recalls.append(recall)

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0

    return avg_precision, avg_recall, f1

def calculate_ceafe(true_clusters, pred_clusters):
    def phi(cluster1, cluster2):
        return len(set(cluster1) & set(cluster2))

    n = len(true_clusters)
    m = len(pred_clusters)

    if n == 0 and m == 0:
        return 1.0, 1.0, 1.0
    if n == 0 or m == 0:
        return 0.0, 0.0, 0.0

    # Simple greedy matching
    used_pred = set()
    total_phi = 0

    for true_cluster in true_clusters:
        best_phi = 0
        best_pred_idx = -1
        for i, pred_cluster in enumerate(pred_clusters):
            if i not in used_pred:
                phi_val = phi(true_cluster, pred_cluster)
                if phi_val > best_phi:
                    best_phi = phi_val
                    best_pred_idx = i
        if best_pred_idx != -1:
            used_pred.add(best_pred_idx)
            total_phi += best_phi

    precision = total_phi / sum(len(cluster) for cluster in pred_clusters) if pred_clusters else 0
    recall = total_phi / sum(len(cluster) for cluster in true_clusters) if true_clusters else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

def calculate_lea(true_clusters, pred_clusters):
    entity_to_true_cluster = {}
    entity_to_pred_cluster = {}

    for cluster in true_clusters:
        for entity in cluster:
            entity_to_true_cluster[entity] = cluster

    for cluster in pred_clusters:
        for entity in cluster:
            entity_to_pred_cluster[entity] = cluster

    all_entities = set(entity_to_true_cluster.keys()) | set(entity_to_pred_cluster.keys())

    total_importance = 0
    total_resolution = 0

    for entity in all_entities:
        true_cluster = entity_to_true_cluster.get(entity, {entity})
        pred_cluster = entity_to_pred_cluster.get(entity, {entity})

        importance = len(true_cluster)
        resolution = len(set(true_cluster) & set(pred_cluster)) / len(set(true_cluster) | set(pred_cluster))

        total_importance += importance
        total_resolution += importance * resolution

    lea_score = total_resolution / total_importance if total_importance > 0 else 0
    return lea_score, lea_score, lea_score

In [ ]:
def evaluate_model(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.data, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return np.array(predictions), np.array(true_labels)

# Get predictions
pred_labels, true_labels = evaluate_model(model, test_loader, device)

# Basic F1 score
f1 = f1_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)

print(f"Basic F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Create clusters for coreference evaluation
def create_clusters_from_predictions(pairs, predictions):
    clusters = []
    entity_to_cluster = {}

    for i, (pair, pred) in enumerate(zip(pairs, predictions)):
        if pred == 1:  # Coreferent
            entity1, entity2 = pair

            cluster1 = entity_to_cluster.get(entity1)
            cluster2 = entity_to_cluster.get(entity2)

            if cluster1 is None and cluster2 is None:
                new_cluster = {entity1, entity2}
                clusters.append(new_cluster)
                entity_to_cluster[entity1] = new_cluster
                entity_to_cluster[entity2] = new_cluster
            elif cluster1 is None:
                cluster2.add(entity1)
                entity_to_cluster[entity1] = cluster2
            elif cluster2 is None:
                cluster1.add(entity2)
                entity_to_cluster[entity2] = cluster1
            elif cluster1 != cluster2:
                # Merge clusters
                merged = cluster1 | cluster2
                clusters.remove(cluster1)
                clusters.remove(cluster2)
                clusters.append(merged)
                for entity in merged:
                    entity_to_cluster[entity] = merged

    return [list(cluster) for cluster in clusters]

# Create predicted clusters
pred_clusters = create_clusters_from_predictions(test_pairs, pred_labels)

# Create true clusters (simple heuristic: same tokens are coreferent)
true_clusters = []
all_entities = set()
for pair in test_pairs:
    all_entities.update(pair)

entity_groups = {}
for entity in all_entities:
    if entity not in entity_groups:
        entity_groups[entity] = [entity]

true_clusters = list(entity_groups.values())

# Calculate metrics
muc_p, muc_r, muc_f1 = calculate_muc(true_clusters, pred_clusters)
b3_p, b3_r, b3_f1 = calculate_bcubed(true_clusters, pred_clusters)
ceafe_p, ceafe_r, ceafe_f1 = calculate_ceafe(true_clusters, pred_clusters)
lea_p, lea_r, lea_f1 = calculate_lea(true_clusters, pred_clusters)

print(f"\nMUC - Precision: {muc_p:.4f}, Recall: {muc_r:.4f}, F1: {muc_f1:.4f}")
print(f"B³ - Precision: {b3_p:.4f}, Recall: {b3_r:.4f}, F1: {b3_f1:.4f}")
print(f"CEAFE - Precision: {ceafe_p:.4f}, Recall: {ceafe_r:.4f}, F1: {ceafe_f1:.4f}")
print(f"LEA - Precision: {lea_p:.4f}, Recall: {lea_r:.4f}, F1: {lea_f1:.4f}")

Basic F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000

MUC - Precision: 0.0000, Recall: 0.0000, F1: 0.0000
B³ - Precision: 1.0000, Recall: 1.0000, F1: 1.0000
CEAFE - Precision: 1.0000, Recall: 0.3425, F1: 0.5102
LEA - Precision: 1.0000, Recall: 1.0000, F1: 1.0000


In [ ]:
# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
    'tokenizer': tokenizer,
    'model_name': model_name
}, '/content/coreference_model.pth')

print("Model saved successfully!")

Model saved successfully!


In [3]:
# !cp /content/drive/MyDrive/coreference/coreference_model.pth /content/.

In [ ]:
# Load the model
checkpoint = torch.load('/content/coreference_model.pth', weights_only=False)

# Initialize model
loaded_model = CoreferenceModel(checkpoint['model_name']).to(device)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_tokenizer = checkpoint['tokenizer']

print("Model loaded successfully!")

# Test the loaded model
loaded_model.eval()
with torch.no_grad():
    test_pair = ("உன்னைப்", "நீ")
    text = f"{test_pair[0]} [SEP] {test_pair[1]}"

    encoding = loaded_tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    outputs = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
    _, predicted = torch.max(outputs.data, 1)

    print(f"Test pair: {test_pair}")
    print(f"Prediction: {'Coreferent' if predicted.item() == 1 else 'Not coreferent'}")

Model loaded successfully!
Test pair: ('உன்னைப்', 'நீ')
Prediction: Coreferent


In [ ]:
len(sentences)

361

In [ ]:
' '.join(sentences[0])

'உன்னைப் பார்த்தால் தெரியவில்லையா ? நீ தப்பி ஓடி ஒளிந்து கொள்ள வந்திருக்கிறவன் என்று நேற்றைக்கே ஊகித்தேன் . இன்றைக்குக் காலையில் உன் சிநேகிதன் , வைத்தியர் மகன் மூலமாக அது ஊர்ஜிதமாயிற்று . அவன் என்ன உளறினான் ? காலையில் எழுந்ததும் காட்டிலே மூலிகை தேட வேண்டும் என்றான் . நான் அழைத்துப் போவதாகச் சொல்லி இங்கே அழைத்துக் கொண்டு வந்தேன் . என்னிடத்தில் காதல் புரிய ஆரம்பித்தான் . உன்னுடைய சிநேகிதன் உன்னை முந்திக் கொண்டு விட்டானே ? ’ என்று சொன்னேன் என்ன சொன்னாய் ? கொஞ்சம் பொறு ; கேட்டுக் கொண்டு வா ! நீ என்னிடம் காதல் புரியத் தொடங்கி விட்டதாகச் சொன்னேன் . அப்போது தான் உன் பேரில் அவனுடைய சந்தேகத்தை வெளியிட்டான் .'

In [12]:
import pandas as pd
import torch
import re
from collections import defaultdict

def create_word_cluster_table(clusters, word_to_id):
    """Create a DataFrame for word cluster visualization"""
    data = []
    for i, cluster in enumerate(clusters):
        cluster_id = f"C{i+1}"
        for word in cluster:
            data.append({
                'Cluster ID': cluster_id,
                'Word': word,
                'Word ID': word_to_id[word]
            })

    return pd.DataFrame(data)

def extract_words_from_tamil_text(text):
    """Extract meaningful words from Tamil text, handling Tamil script properly"""
    # Remove punctuation and split by spaces
    # This regex preserves Tamil characters and removes common punctuation
    words = re.findall(r'[\u0B80-\u0BFF]+|[a-zA-Z]+', text)

    # Filter out very short words (single characters) that might be punctuation
    words = [word.strip() for word in words if len(word.strip()) > 1]

    return words

def predict_coreference(model, tokenizer, device, word1, word2):
    """Predict if two words are coreferent"""
    if model is None:
        # Fallback: simple string similarity for demo
        if word1.lower() == word2.lower():
            return 0.9
        elif word1.lower() in word2.lower() or word2.lower() in word1.lower():
            return 0.6
        else:
            return 0.3

    # Format input as the model expects
    text = f"{word1} [SEP] {word2}"

    try:
        encoding = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            coreference_prob = probabilities[0][1].item()  # Probability of being coreferent

        return coreference_prob

    except Exception as e:
        print(f"Error predicting coreference: {e}")
        return 0.0

def create_word_clusters_from_tamil_sentence(text, model, tokenizer, device, threshold=0.7):
    """Create coreference clusters from Tamil sentence input - returns word clusters"""

    # Extract words from the text
    words = extract_words_from_tamil_text(text)

    if not words:
        return [], {}

    print(f"Extracted words: {words}")

    # Create word mapping
    unique_words = list(set(words))
    word_to_id = {word: f"W{i+1}" for i, word in enumerate(unique_words)}

    # Track word positions in original text
    word_positions = defaultdict(list)
    for i, word in enumerate(words):
        word_positions[word].append(i)

    # Create clusters using coreference prediction
    clusters = []
    used_words = set()

    for i, word1 in enumerate(unique_words):
        if word1 in used_words:
            continue

        current_cluster = [word1]
        used_words.add(word1)

        for j, word2 in enumerate(unique_words[i+1:], i+1):
            if word2 in used_words:
                continue

            # Predict coreference between words
            prob = predict_coreference(model, tokenizer, device, word1, word2)
            # print(f"Coreference probability between '{word1}' and '{word2}': {prob:.3f}")

            if prob > threshold:
                current_cluster.append(word2)
                used_words.add(word2)

        # Only keep clusters with more than one word
        if len(current_cluster) > 1:
            clusters.append(current_cluster)

    return clusters, word_to_id

def get_coreference_clusters_tamil_words(text, model=None, tokenizer=None, device=None, threshold=0.7):
    """
    Main function: Given a Tamil sentence, return word coreference clusters.
    Returns both clusters and a DataFrame for visualization.

    Args:
        text (str): Tamil sentence
        model: Trained coreference model (optional)
        tokenizer: Model tokenizer (optional)
        device: PyTorch device (optional)
        threshold (float): Coreference probability threshold

    Returns:
        clusters (list): List of word clusters
        cluster_df (DataFrame): DataFrame for visualization
    """

    # Ensure input is a valid string
    if not isinstance(text, str) or not text.strip():
        return [], pd.DataFrame()

    print(f"Input text: {text}")

    # Set default device if not provided
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Run cluster creation
    clusters, word_to_id = create_word_clusters_from_tamil_sentence(
        text=text,
        model=model,
        tokenizer=tokenizer,
        device=device,
        threshold=threshold
    )

    # Build DataFrame for visualization
    if clusters:
        cluster_df = create_word_cluster_table(clusters, word_to_id)
    else:
        cluster_df = pd.DataFrame(columns=['Cluster ID', 'Word', 'Word ID'])

    return clusters, cluster_df

def load_coreference_model():
    """Load the trained coreference model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    try:
        checkpoint = torch.load('/content/coreference_model.pth', weights_only=False)

        # You'll need to import your CoreferenceModel class here
        # from your_model_file import CoreferenceModel
        model = CoreferenceModel(checkpoint['model_name']).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        tokenizer = checkpoint['tokenizer']
        model.eval()

        print("Model loaded successfully!")
        return model, tokenizer, device

    except Exception as e:
        print(f"Could not load model: {e}")
        print("Running in demo mode without trained model...")
        return None, None, device

In [13]:
# Load model (or use None for demo)
model, tokenizer, device = load_coreference_model()

# Example Tamil sentences (you can replace with sentences from your training data)
test_sentences = [' '.join(sentences[0])]
print(test_sentences[0])
for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*60}")
    print(f"Test Sentence {i}:")
    print(f"{'='*60}")

    clusters, cluster_df = get_coreference_clusters_tamil_words(
        text=sentence,
        model=model,
        tokenizer=tokenizer,
        device=device,
        threshold=0.6  # Lower threshold for demo
    )

    print(f"\nFound {len(clusters)} clusters:")

    if clusters:
        for j, cluster in enumerate(clusters, 1):
            print(f"  Cluster {j}: {cluster}")
    else:
        print("  No coreference clusters found.")

    if not cluster_df.empty:
        print(f"\nCluster DataFrame:")
        print(cluster_df.to_string(index=False))

    print(f"\n{'='*60}")

Model loaded successfully!
உன்னைப் பார்த்தால் தெரியவில்லையா ? நீ தப்பி ஓடி ஒளிந்து கொள்ள வந்திருக்கிறவன் என்று நேற்றைக்கே ஊகித்தேன் . இன்றைக்குக் காலையில் உன் சிநேகிதன் , வைத்தியர் மகன் மூலமாக அது ஊர்ஜிதமாயிற்று . அவன் என்ன உளறினான் ? காலையில் எழுந்ததும் காட்டிலே மூலிகை தேட வேண்டும் என்றான் . நான் அழைத்துப் போவதாகச் சொல்லி இங்கே அழைத்துக் கொண்டு வந்தேன் . என்னிடத்தில் காதல் புரிய ஆரம்பித்தான் . உன்னுடைய சிநேகிதன் உன்னை முந்திக் கொண்டு விட்டானே ? ’ என்று சொன்னேன் என்ன சொன்னாய் ? கொஞ்சம் பொறு ; கேட்டுக் கொண்டு வா ! நீ என்னிடம் காதல் புரியத் தொடங்கி விட்டதாகச் சொன்னேன் . அப்போது தான் உன் பேரில் அவனுடைய சந்தேகத்தை வெளியிட்டான் .

Test Sentence 1:
Input text: உன்னைப் பார்த்தால் தெரியவில்லையா ? நீ தப்பி ஓடி ஒளிந்து கொள்ள வந்திருக்கிறவன் என்று நேற்றைக்கே ஊகித்தேன் . இன்றைக்குக் காலையில் உன் சிநேகிதன் , வைத்தியர் மகன் மூலமாக அது ஊர்ஜிதமாயிற்று . அவன் என்ன உளறினான் ? காலையில் எழுந்ததும் காட்டிலே மூலிகை தேட வேண்டும் என்றான் . நான் அழைத்துப் போவதாகச் சொல்லி இங்கே அழைத்துக் கொண்டு வந்தேன் . என்னிட

In [ ]:
# Example Tamil sentences (you can replace with sentences from your training data)
test_sentences = [' '.join(sentences[1])]
print(test_sentences[0])
for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*60}")
    print(f"Test Sentence {i}:")
    print(f"{'='*60}")

    clusters, cluster_df = get_coreference_clusters_tamil_words(
        text=sentence,
        model=model,
        tokenizer=tokenizer,
        device=device,
        threshold=0.6  # Lower threshold for demo
    )

    print(f"\nFound {len(clusters)} clusters:")

    if clusters:
        for j, cluster in enumerate(clusters, 1):
            print(f"  Cluster {j}: {cluster}")
    else:
        print("  No coreference clusters found.")

    if not cluster_df.empty:
        print(f"\nCluster DataFrame:")
        print(cluster_df.to_string(index=False))

    print(f"\n{'='*60}")

இல்லாவிட்டால் ? இங்கே என் முன்னால் சொல்லியிருந்தால் அந்தச் சேற்றுக் குழியில் தூக்கிப் போட்டிருப்பேன் . அதனால் என்ன ? சேற்றை அலம்பிக்கொள்ளக் கடலில் ஏராளமாய்த் தண்ணீர் இருக்கிறதே ! நீ விழுந்த புதை சேற்றுக்குழியில் மாடு , குதிரை எல்லாம் முழுகிச் செத்திருக்கின்றன . யானையைக் கூட அது விழுங்கி விடும் ! வந்தியத்தேவனுடைய உடம்பு சிலிர்த்தது . அவனை அந்தப் படுகுழி கொஞ்சமாகக் கீழே இழுத்துக் கொண்டிருந்த போது ஏற்பட்ட உணர்ச்சியை நினைத்துக் கொண்டான் . இவள் மட்டும் வந்து கரையேற்றியிராவிட்டால் , இத்தனை நேரம் . அதை நினைத்தபோது அவன் உடம்பெல்லாம் நடுங்கிற்று . " சேந்தன் அமுதன் என்னைப்பற்றி இன்னும் என்ன சொன்னான் ? ” என்று பூங்குழலி கேட்டாள் . நீ அவனுடைய மாமன் மகள் என்று சொன்னான் .

Test Sentence 1:
Input text: இல்லாவிட்டால் ? இங்கே என் முன்னால் சொல்லியிருந்தால் அந்தச் சேற்றுக் குழியில் தூக்கிப் போட்டிருப்பேன் . அதனால் என்ன ? சேற்றை அலம்பிக்கொள்ளக் கடலில் ஏராளமாய்த் தண்ணீர் இருக்கிறதே ! நீ விழுந்த புதை சேற்றுக்குழியில் மாடு , குதிரை எல்லாம் முழுகிச் செத்திருக்கின்றன . யானையைக் கூட அது விழுங்கி விடும் ! வந்திய

In [ ]:
# Example Tamil sentences (you can replace with sentences from your training data)
test_sentences = [' '.join(sentences[2])]
print(test_sentences[0])
for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*60}")
    print(f"Test Sentence {i}:")
    print(f"{'='*60}")

    clusters, cluster_df = get_coreference_clusters_tamil_words(
        text=sentence,
        model=model,
        tokenizer=tokenizer,
        device=device,
        threshold=0.6  # Lower threshold for demo
    )

    print(f"\nFound {len(clusters)} clusters:")

    if clusters:
        for j, cluster in enumerate(clusters, 1):
            print(f"  Cluster {j}: {cluster}")
    else:
        print("  No coreference clusters found.")

    if not cluster_df.empty:
        print(f"\nCluster DataFrame:")
        print(cluster_df.to_string(index=False))

    print(f"\n{'='*60}")

1650 இல் அவன் டென்மார்க்கில் நிதி அமைச்சகத்தில் ஆலோசகராக தனது தொழிலைத் தொடங்கினான் . அமைச்சரான பிறகு அவன் அம்ஸ்டர்டாமுக்குத் திரும்பி , இந்த நகரத்தின் ஒரு வகையான தலைமை மேயராகப் பதவி வகித்தான் . தனது சகோதரர் கார்னெலிஸின் மரணத்திற்குப் பிறகு , மோரிஸ் குடியரசுக் கட்சியின் வலுவான தலைவரானான் . தனது மரணம் வரை அவன் இந்தப் பதவியை வகித்தான் .

Test Sentence 1:
Input text: 1650 இல் அவன் டென்மார்க்கில் நிதி அமைச்சகத்தில் ஆலோசகராக தனது தொழிலைத் தொடங்கினான் . அமைச்சரான பிறகு அவன் அம்ஸ்டர்டாமுக்குத் திரும்பி , இந்த நகரத்தின் ஒரு வகையான தலைமை மேயராகப் பதவி வகித்தான் . தனது சகோதரர் கார்னெலிஸின் மரணத்திற்குப் பிறகு , மோரிஸ் குடியரசுக் கட்சியின் வலுவான தலைவரானான் . தனது மரணம் வரை அவன் இந்தப் பதவியை வகித்தான் .
Extracted words: ['இல்', 'அவன்', 'டென்மார்க்கில்', 'நிதி', 'அமைச்சகத்தில்', 'ஆலோசகராக', 'தனது', 'தொழிலைத்', 'தொடங்கினான்', 'அமைச்சரான', 'பிறகு', 'அவன்', 'அம்ஸ்டர்டாமுக்குத்', 'திரும்பி', 'இந்த', 'நகரத்தின்', 'ஒரு', 'வகையான', 'தலைமை', 'மேயராகப்', 'பதவி', 'வகித்தான்', 'தனது', 'சகோதரர்', 'கார்னெலிஸின

In [14]:
# Example Tamil sentences (you can replace with sentences from your training data)
test_sentences = ['அருண் ஒரு புத்தகம் வாங்கினான். அவன் அதை படித்தான்.']
print(test_sentences[0])
for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*60}")
    print(f"Test Sentence {i}:")
    print(f"{'='*60}")

    clusters, cluster_df = get_coreference_clusters_tamil_words(
        text=sentence,
        model=model,
        tokenizer=tokenizer,
        device=device,
        threshold=0.6  # Lower threshold for demo
    )

    print(f"\nFound {len(clusters)} clusters:")

    if clusters:
        for j, cluster in enumerate(clusters, 1):
            print(f"  Cluster {j}: {cluster}")
    else:
        print("  No coreference clusters found.")

    if not cluster_df.empty:
        print(f"\nCluster DataFrame:")
        print(cluster_df.to_string(index=False))

    print(f"\n{'='*60}")

அருண் ஒரு புத்தகம் வாங்கினான். அவன் அதை படித்தான்.

Test Sentence 1:
Input text: அருண் ஒரு புத்தகம் வாங்கினான். அவன் அதை படித்தான்.
Extracted words: ['அருண்', 'ஒரு', 'புத்தகம்', 'வாங்கினான்', 'அவன்', 'அதை', 'படித்தான்']

Found 0 clusters:
  No coreference clusters found.



In [15]:
# Example Tamil sentences (you can replace with sentences from your training data)
test_sentences = ['குமார் பள்ளிக்குச் சென்றான். அவன் நண்பர்களை பார்த்தான்.']
print(test_sentences[0])
for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*60}")
    print(f"Test Sentence {i}:")
    print(f"{'='*60}")

    clusters, cluster_df = get_coreference_clusters_tamil_words(
        text=sentence,
        model=model,
        tokenizer=tokenizer,
        device=device,
        threshold=0.6  # Lower threshold for demo
    )

    print(f"\nFound {len(clusters)} clusters:")

    if clusters:
        for j, cluster in enumerate(clusters, 1):
            print(f"  Cluster {j}: {cluster}")
    else:
        print("  No coreference clusters found.")

    if not cluster_df.empty:
        print(f"\nCluster DataFrame:")
        print(cluster_df.to_string(index=False))

    print(f"\n{'='*60}")

குமார் பள்ளிக்குச் சென்றான். அவன் நண்பர்களை பார்த்தான்.

Test Sentence 1:
Input text: குமார் பள்ளிக்குச் சென்றான். அவன் நண்பர்களை பார்த்தான்.
Extracted words: ['குமார்', 'பள்ளிக்குச்', 'சென்றான்', 'அவன்', 'நண்பர்களை', 'பார்த்தான்']

Found 1 clusters:
  Cluster 1: ['அவன்', 'சென்றான்']

Cluster DataFrame:
Cluster ID     Word Word ID
        C1     அவன்      W3
        C1 சென்றான்      W6



In [ ]:
!mkdir -p /content/drive/MyDrive/coreference/
!cp /content/coreference_model.pth /content/drive/MyDrive/coreference/